In [1]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Activation, Embedding, Conv1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from datetime import datetime


from dataprocessing.find_top_movie_with_sequence import find_top_dataset
from dataprocessing.label_and_feature import getSortedClassContents, checkConsists, buildFeature, saveClassesToFile
from dataprocessing.time import checkingTimeDifferent
from recom_model import trainWithBidirectional,trainWithEmbeddingDense

[[[[0, 0, 2]], 4], [[[0, 0, 1]], 1], [[[0, 0, 3]], 5], [[[0, 0, 4]], 7], [[[0, 3, 1]], 3]]


In [2]:
MIN_CONTENTS_ON_USER = 250
MAX_DAYS = 7
MAX_SEQUENCE = 1
DATA_DIR = "data"

recommandation_df = pd.read_csv('{}/data1.csv'.format(DATA_DIR)).sort_values(by=['time'])

In [3]:
sortedClassContents = getSortedClassContents(recommandation_df, MIN_CONTENTS_ON_USER)
saveClassesToFile(sortedClassContents)
lenSortedClassContents = len(sortedClassContents)
print(lenSortedClassContents)

[0, 1, 15, 23, 39, 69, 79, 162, 169, 202, 205, 213, 252, 255, 306, 308, 312, 322, 369, 371, 379, 434, 448, 455, 478, 479, 480, 481, 492, 507, 513, 535, 548, 623, 626, 640, 644, 657, 682, 683, 697, 700, 752, 772, 776, 777, 781, 824, 832, 839, 883, 913, 936, 960, 964, 977, 989, 993, 1006, 1020, 1026, 1115, 1137, 1185, 1187, 1193, 1239, 1242, 1256, 1269, 1304, 1310, 1397, 1415, 1420, 1428, 1429, 1433, 1436, 1437, 1490, 1498, 1506, 1512, 1521, 1534, 1554, 1559, 1567, 1578, 1579, 1589, 1591, 1599, 1602, 1606, 1607, 1609, 1619, 1620, 1624, 1641, 1652, 1659, 1677, 1700, 1716, 1719, 1756, 1758, 1774, 1777, 1781, 1788, 1790, 1791, 1795, 1798, 1806, 1807, 1813, 1819, 1823, 1826, 1830, 1833, 1834, 1836, 1838, 1839, 1840, 1845, 1847, 1853, 1855, 1862, 1863, 1872, 1873, 1876, 1877, 1879, 1885, 1896, 1910, 1912, 1913, 1927, 1928, 1935, 1936]
151


In [4]:
def saveDatas(datas):
    features = []
    labels = []
    for feature, label in datas:
        features.append(feature)
        labels.append(label)

    a_file = open('{}/backup_features.txt'.format(DATA_DIR), "w")
    for row in features:
        np.savetxt(a_file, row[0])
    a_file.close()

    a_file = open('{}/backup_labels.txt'.format(DATA_DIR), "w")
    np.savetxt(a_file, labels)
    a_file.close()

In [5]:
def build_training_data(data_frame):
    # datetime object containing current date and time
    now = datetime.now()

    print("start =", now)

    visitors_df = data_frame['visitor'].drop_duplicates()
    maxItem = data_frame['Content'].max()
    training_data = []
    for index, item in visitors_df.iteritems():
        video = data_frame[data_frame['visitor'] == item]
        if video.size > 1:
            tempContents = []
            indexContent = 0
            for index, item in video['Content'].iteritems():
                if item not in sortedClassContents:
                    continue
                if len(tempContents) > MAX_SEQUENCE:
                    tempContents = tempContents[1:]
                    continue
                if checkConsists(item, tempContents):
                    continue
                if len(tempContents) > 0:
                    feature = buildFeature(tempContents, sortedClassContents, MAX_SEQUENCE)
                    label = sortedClassContents.index(item)
                    training_data.append([[feature], label])
                tempContents.append(item)
                indexContent += 1

    now = datetime.now()
    print("end =", now)
    return training_data

In [6]:
data1 = pd.read_csv('{}/data1.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = build_training_data(data1)
print(len(training_data))
saveDatas(training_data)

start = 2020-11-16 10:07:44.667945
end = 2020-11-16 10:37:31.659633
17406


In [ ]:
data2 = pd.read_csv('{}/data2.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = np.concatenate((training_data, build_training_data(data2)))

In [ ]:
print('{}.before', len(training_data))
data3 = pd.read_csv('{}/data3.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = np.concatenate((training_data, build_training_data(data3)))
print(len(training_data))
saveDatas(training_data)

In [ ]:
data4 = pd.read_csv('{}/data4.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = np.concatenate((training_data, build_training_data(data4)))
print(len(training_data))
saveDatas(training_data)

In [ ]:
data5 = pd.read_csv('{}/data5.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = np.concatenate((training_data, build_training_data(data5)))
print(len(training_data))
saveDatas(training_data)

In [ ]:
data6 = pd.read_csv('{}/data6.csv'.format(DATA_DIR)).sort_values(by=['time'])
training_data = np.concatenate((training_data, build_training_data(data6)))
print(len(training_data))
saveDatas(training_data)

In [ ]:
validation_data = find_top_dataset(training_data)
print("training size: ", len(training_data))
print("validation size: ", len(validation_data))
# print(training_data)
print("-------------------------")
# print(validation_data)
random.shuffle(training_data)


In [ ]:
features = []
labels = []
for feature, label in training_data:
    features.append(feature)
    labels.append(label)

In [ ]:
X = np.array(features)
Y = np.array(labels)
print(X.shape)



In [ ]:
features_val = []
labels_val = []
for feature, label in validation_data:
    features_val.append(feature)
    labels_val.append(label)

X_val = np.array(features_val)
Y_val = np.array(labels_val).astype(np.float32)

In [ ]:
# for cnn
X = np.squeeze(X, axis=1)
print(X.shape)
model = Sequential()   
model.add(Dense(1024, activation="relu", input_shape=(X.shape[1:])))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(lenSortedClassContents))
model.add(Activation("softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


In [ ]:
# for embedding Dense
X = np.squeeze(X, axis=1)
model = trainWithEmbeddingDense(MAX_SEQUENCE, lenSortedClassContents)

In [ ]:
now = datetime.now()
name = "testing-1{}".format(now)
tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
history = model.fit(X, Y, epochs=256, batch_size=64, verbose=1,validation_data=(X_val, Y_val), callbacks=[tensorboard])

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
plot_graphs(history, 'loss')

In [ ]:
plot_graphs(history, 'val_loss')

In [ ]:
plot_graphs(history, 'val_accuracy')

In [ ]:
def get5TopPredict(predict):
    predicts = predict[0].argsort()[-5:][::-1]
    for i in predicts:
        print(sortedClassContents[i])
        

In [ ]:
      
def predictDense(first):
    predict = model.predict([[sortedClassContents.index(first)]])
    get5TopPredict(predict)
    return sortedClassContents[np.argmax(predict[0])]


In [ ]:
predictResult = predictDense(1506)
print(predictResult)

In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1,6], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "model"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)


In [ ]:
MODEL_DIR = "model"
model.save(MODEL_DIR) 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
print(input_details[0]["index"])
for i in range(TEST_CASES):
  expected = model.predict([[i]])
  interpreter.set_tensor( input_details[0]["index"], np.array([[i]]).astype(np.float32))
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()


In [ ]:
open("tf_model/converted_model.tflite", "wb").write(tflite_model)

In [ ]:
interpreter.set_tensor( input_details[0]["index"], np.array([[sortedClassContents.index(700)]]).astype(np.float32))
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]["index"])
print(sortedClassContents[np.argmax(result)])